<a href="https://colab.research.google.com/github/radf0001/NLP/blob/main/Pra%CC%81ctica_1_Extraccio%CC%81n_de_Informacio%CC%81n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="background:#5D6D7E;padding:20px;color:#ffffff;margin-top:10px;">

# NLP - Práctica 1 ( Extracción de Información)

## Profesora: Lisibonny Beato
### Período 3-2022-2023</div>


In [ ]:
!python -m pip install textacy
!python -m pip install visualise_spacy_tree
!python -m spacy download en_core_web_trf
!python -m pip install spacy_transformers
!python  -m spacy download en_core_web_sm

In [ ]:
# Importando algunas librerias

import pandas as pd
import re
import spacy
import textacy
import glob
import visualise_spacy_tree
from spacy.matcher import Matcher
import os
import sys
import en_core_web_trf
import spacy_transformers
import re
import nltk
import string

<div style="background:#ff6242;padding:20px;color:#ffffff;margin-top:10px;">
<b>Para esta práctica estarán utilizando el corpus WNUT16, el cual encontrarán en la carpeta de dataset de nuestro curso. Para más información sobre este corpus ver el siguiente paper: https://aclanthology.org/W16-3919.pdf
<br />
<br />
Aparte del código, debe proveer una interpretación para cada tarea y un análisis para cada resultado obtenido que así lo amerite.</b>
</div>


## 1. Nivel Básico
### Puntuación máxima de la tarea: 5 puntos
#### Limpieza y preparación de los datos, Extracción de palabras clave (KPE),  y reconocimiento de entidades relevantes (NER) con modelos heurísticos y pre-entrenados para  reconocer al menos 3 tipos de entidades.

In [ ]:
# load english language model
nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])

In [ ]:
pd.set_option('display.max_colwidth', 2000)
pd.options.display.max_rows=100

In [ ]:
# Cargando Dataset
df = pd.DataFrame(columns=["Oracion"])
i = 0
oracion = ""
with open("/content/wnut16_test.txt") as bookfile:
    for line in bookfile:
        fields = line.split("\t")
        if(fields[0] != "\n"):
          if(oracion == "" or fields[0] in string.punctuation):
            oracion += fields[0]
          else:
            oracion += " " + fields[0]
        else:
          df.loc[i, "Oracion"] = oracion;
          oracion = ""
          i += 1

In [ ]:
df

In [ ]:
def limpiar(text):
    # Quitar numero de parrafo
    text = re.sub('[0-9]+.\t', '', str(text))
    text = re.sub('[0-9]+.', '', str(text))
    # Quitando algunos caracteres
    text = re.sub('\n', '', str(text))
    text = re.sub('\n ', '', str(text))
    text = re.sub('-', '', str(text))
    text = re.sub('- ', '', str(text))
    text = re.sub('"', '', str(text))
    text = re.sub("'s", '', str(text))

    return text

def longitud(text):
  return len(text.split())

In [ ]:
df['Oracion_Limpia']=df['Oracion'].apply(limpiar)
df['Longitud']=df['Oracion_Limpia'].apply(longitud)

In [ ]:
df

In [ ]:
count=0
for i in range(len(df)):
    if df.loc[i,'Longitud']==0:
        count+=1
print(count)

### Extraccion de KPES

In [ ]:
# Vamos a tomar solo las primeras 4000 oraciones, ya que la funcion que convierte a documento de spacy no permite
# mas de 1000000 de caracteres para el documento.
oraciones_sample=df['Oracion_Limpia'][0:4000]

In [ ]:
all_words = ' '.join([word for word in oraciones_sample])
all_words=all_words.lower()
all_words

"new orleans mother  day parade shooting. one of the people hurt was a yearold girl. what the hell is wrong with people? rt @hxranspizza: going into school tomorrow like #kca #voteuk http://t.co/vvkoeemjmx may e just a smile in your heart eily countdownbegins #pushawardslizquens i could so do thursday club right now @therealdaftbear albert nobbs( glenn close)is a woman living as a man in order to find work in the harsh environment of hcentury ireland rt @asiacrew: louis' interview with the sun about his own record label productions limited #proudoflouis http://t.co/hdswyedbio injured in minneapolis nightclub shooting; vikings player among victims: http://bring.mn/ so happy moving into my new place on tuesday! rt @aldenrichards happy h weeksary sa ating lahat !!! #aldubourmissionislove #ikawyung tangi kung hiling sa h bday ko i drive by that motel almost every night. #mesashooting apple macbook pro a laptop mdl/a( june,  full read by ebay http://t.co/gqmuf http://t.co/eqmogqqabk tuff cu

In [ ]:
# Voy a convertir el string con todos los speeches a un formato de documento spacy
speeches=textacy.make_spacy_doc(all_words, lang='en_core_web_sm')

In [ ]:
# Con este linea pedimos que se obtengan los mejores N KPEs desde el texto
textacy.extract.keyterms.textrank(speeches,topn=10)

[('monday https://t.co/nkffju rt @pojoktweet silakan rt manual tweet ini',
  0.04756312546183921),
 ('womeninfiction rt @juliaruegg rt', 0.04584133119398151),
 ('rt http://t.co/xhfoeufj rt @adventurefui', 0.04584006405199604),
 ('rt rt @ezimuoh', 0.04581827016600244),
 ('rt rt @official_b', 0.04581662965181931),
 ('new job tomorrow bruh rt @ohsaklapfriend', 0.02727493851389692),
 ('rt @aldenrichards happy h weeksary sa', 0.027191124443839212),
 ('cest quand ya un mariage haitien que je mette un costard et je tape mon meilleur kompa rt @fiveshinki',
  0.026818014572063488),
 ('school tomorrow https://t.co/xspjwylq rt @gatorsfb', 0.026537273474964714),
 ('school tomorrow http://t.co/txuzonrssz rt @exoffical',
  0.026537273474964714)]

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Utilice esta celda para colocar comentarios en el notebook, cuando lo estime necesario. Copiela varias veces donde considere.
</div>

## 2. Nivel Intermedio
### Puntuación máxima de la tarea: 2 puntos
#### Entrenar un modelo propio de NER mediante el pipeline de Spacy

## 3. Nivel Avanzado
### Puntuación máxima de la tarea: 3 puntos
#### Entrenar un modelo de NER mediante el uso de Conditional Random Fields. Recuerde ir más allá que donde llega el ejemplo al respecto provisto en clases.